In [ ]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

## Cyclegan

In [ ]:
from fastai.conv_learner import *
from fastai.dataset import *

In [ ]:
from cgan.options.train_options import *

from cgan.data.data_loader import CreateDataLoader
from cgan.models.models import create_model
import cv2

In [ ]:
import warnings
warnings.simplefilter("ignore")

In [ ]:
torch.cuda.set_device(0)

In [ ]:
'''from google_images_download import google_images_download   #importing the library

response = google_images_download.googleimagesdownload()   #class instantiation

arguments = {"keywords":"Squidward","limit":100,"print_urls":True}   #creating list of arguments
paths = response.download(arguments)   #passing the arguments to the function
print(paths)   #printing absolute paths of the downloaded images'''

In [ ]:
'''import requests

url = 'https://people.eecs.berkeley.edu/~taesung_park/CycleGAN/datasets/horse2zebra.zip'
r = requests.get(url, allow_redirects=True)
open('horse2zebra.zip', 'wb').write(r.content)'''

In [ ]:
def scale_match(h, w, targ):
    sh,sw,_ = targ.shape
    rat = max(h/sh,w/sw); rat
    res = cv2.resize(targ, (int(sw*rat), int(sh*rat)))
    return res[:h,:w]

In [ ]:
path1 = 'data/downloads/squidward/'
os.listdir(path1)[:4]

In [ ]:
def get_shape_mean(path):
    sumx = 0
    sumy = 0
    count = 0
    
    for fname in os.listdir(path):
        if fname[-3:] == 'jpg':
            style_img = open_image(path + fname)
            style_img
            sumx += style_img.shape[0]
            sumy += style_img.shape[1]
            count += 1

    return sumx/count, sumy/count 

In [ ]:
def set_shape_mean(path):
    mean_h, mean_w = 224,224#get_shape_mean(path1)

    for fname in os.listdir(path):
        if fname[-3:] == 'jpg':
            style_img = open_image(path + fname)
            style = scale_match(int(mean_h), int(mean_w), style_img)
            cv2.imwrite(f'{path}/converted/converted_{fname}',cv2.cvtColor(style[:, :, :]*255, cv2.COLOR_BGR2RGB))

In [ ]:
#set_shape_mean(path1)

In [ ]:
opt = TrainOptions().parse(['--dataroot', '/data/shapiro2squidward', '--nThreads', '8', '--no_dropout',
                           '--niter', '100', '--niter_decay', '100', '--name', 'nodrop', '--gpu_ids', '0'])

In [ ]:
data_loader = CreateDataLoader(opt)
dataset = data_loader.load_data()
dataset_size = len(data_loader)
dataset_size

In [ ]:
model = create_model(opt)

In [ ]:
opt.niter, opt.niter_decay

In [ ]:
opt.niter=20
opt.niter_decay=10

In [ ]:
total_steps = 0

for epoch in range(opt.epoch_count, opt.niter + opt.niter_decay + 1):
    epoch_start_time = time.time()
    iter_data_time = time.time()
    epoch_iter = 0

    for i, data in tqdm(enumerate(dataset)):
        iter_start_time = time.time()
        if total_steps % opt.print_freq == 0: t_data = iter_start_time - iter_data_time
        total_steps += opt.batchSize
        epoch_iter += opt.batchSize
        model.set_input(data)
        model.optimize_parameters()

        if total_steps % opt.display_freq == 0:
            save_result = total_steps % opt.update_html_freq == 0

        if total_steps % opt.print_freq == 0:
            errors = model.get_current_errors()
            t = (time.time() - iter_start_time) / opt.batchSize

        if total_steps % opt.save_latest_freq == 0:
            print('saving the latest model (epoch %d, total_steps %d)' % (epoch, total_steps))
            model.save('latest')

        iter_data_time = time.time()
    if epoch % opt.save_epoch_freq == 0:
        print('saving the model at the end of epoch %d, iters %d' % (epoch, total_steps))
        model.save('latest')
        model.save(epoch)

    print('End of epoch %d / %d \t Time Taken: %d sec' %
          (epoch, opt.niter + opt.niter_decay, time.time() - epoch_start_time))
    model.update_learning_rate()

In [ ]:
def show_img(im, ax=None, figsize=None):
    if not ax: fig,ax = plt.subplots(figsize=figsize)
    ax.imshow(im)
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
    return ax

In [ ]:
def get_one(data):
    model.set_input(data)
    model.test()
    return list(model.get_current_visuals().values())

In [ ]:
model.save(201)

In [ ]:
test_ims = []
for i,o in enumerate(dataset):
    if i>10: break
    test_ims.append(get_one(o))

In [ ]:
def show_grid(ims):
    fig,axes = plt.subplots(2,3,figsize=(9,6))
    for i,ax in enumerate(axes.flat): show_img(ims[i], ax);
    fig.tight_layout()

In [ ]:
for i in range(8): show_grid(test_ims[i])